# `tensorflow.keras` sweep across pre-trained models


This notebook demonstrates the use of (1) well-known CNN architectures, (2) pretrained weights on `RadImageNet` in a multiclassification task, (3) `wandb.ai` sweeps in comparing the pre-trained architectures.

References:

* The pretrained weights published by [Mei et al (2022)](https://pubs.rsna.org/doi/10.1148/ryai.210315) in their github [link](https://github.com/BMEII-AI/RadImageNet).

* The github repository form the BAGLS team contained in this github [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "05-pretrained-sweep.ipynb"

PROJECT_NAME = 'bagls-sh-project'
GROUP_NAME = 'pretrained-models-sweep-3'
METRICS_TABLE_NAME = 'metrics_table'

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# manage logs
import logging

logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


In [4]:
# tf loader
import tensorflow as tf
print("TF: ", tf.__version__)
from tensorflow.keras.models import load_model
from tensorflow import keras

TF:  2.6.0


In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14511411650604992723
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 793061614861425578
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 12624532780810070682
physical_device_desc: "device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 2
  numa_node: 1
  links {
    link {
      device_id: 3
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 1

2022-12-04 07:57:37.864485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 07:57:40.706030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-04 07:57:40.707209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-04 07:57:40.707903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci

In [6]:
import config
configs = config.nb_configs
configs.update(config.pt_configs)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=None,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=None, 
    rescale=configs["rescale"],
)

In [8]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 50018 images belonging to 2 classes.
Found 2632 images belonging to 2 classes.
Found 2300 images belonging to 2 classes.


In [9]:
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras import losses

In [10]:
def define_model(model_arch, weights):
    conv_base = model_arch(
        weights=weights,
        include_top=configs["include_top"],
        input_shape=configs["input_shape"],
        pooling=configs["pooling"],
    )
    # freeze conv base weights
    conv_base.trainable = False
    x = conv_base.output
    
    # layers at this stage are arbitrary
    # can be subjected to hyperparam tuning
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs["dropout_rate"])(x)
    outputs = keras.layers.Dense(units=2, activation="softmax")(x)
    model = keras.Model(inputs=conv_base.input, outputs=outputs)
    return model

In [11]:
from interpretation import ValLog, GRADCamLogger
import utils
from tensorflow.keras import metrics
import custom_metrics

def train(model_name):
#     GRADCAM_LAYER_NAME = config.pt_gradcam_layers[model_name]    
    # alternatively, pass none to use a function that searches for last conv layer
    GRADCAM_LAYER_NAME = None
    model_arch = config.pt_models[model_name]
    weights = config.pt_weights[model_name]
    
    tf.keras.backend.clear_session()
    
    thresh = configs["thresh"]
    metrics_dict = {
        "ACC":  metrics.CategoricalAccuracy(name="ACC"),
        "AUC-ROC": custom_metrics.MulticlassAUC(name='ROC', curve='ROC', pos_label=1),
        "AUC-PR": custom_metrics.MulticlassAUC(name='PR', curve='PR', pos_label=1),
        "TP": custom_metrics.MulticlassTruePositives(name="TP", pos_label=1, thresholds=thresh),
        "TN": custom_metrics.MulticlassTrueNegatives(name="TN", pos_label=1, thresholds=thresh),
        "FP": custom_metrics.MulticlassFalsePositives(name="FP", pos_label=1, thresholds=thresh),
        "FN": custom_metrics.MulticlassFalseNegatives(name="FN", pos_label=1, thresholds=thresh),
    }


    # opt = optimizers.Adam(learning_rate=1e-06)
    opt = optimizers.Adam()
    met = list(metrics_dict.values())

    model = define_model(model_arch, weights)
    model.compile(
        loss=losses.CategoricalCrossentropy(),
        optimizer=opt,
        metrics=met,
    )
    # initialize run
    run = wandb.init(
        project=PROJECT_NAME, 
        group=GROUP_NAME,
        name=model_name,
        job_type='train',
        config=configs, 
    )

    wandb_callback = wandb.keras.WandbCallback(
        monitor="val_ROC",
        mode="max",
        save_model=True,
        save_graph=True,
        compute_flops=True,
    )

    callbacks = [
        wandb_callback,
        ValLog(generator=val_dataset, num_log_batches=1),
        GRADCamLogger(generator=test_dataset, 
                      layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    ]
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=configs["epochs"], 
        shuffle=True,
        callbacks=callbacks,
#         verbose=0,
    )
    run.finish()


In [12]:
for model_name in config.pt_models.keys():
#     resumption for inceptionresv2
#     if model_name != 'InceptionResNetV2':
#         continue
    print(model_name)
    train(model_name)

InceptionResNetV2


2022-12-04 07:58:23.211430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-04 07:58:23.212201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-04 07:58:23.212948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 10419 MB memory:  -> device: 2, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1
2022-12-04 07:58:23.213650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 10419 MB memory:  -> device: 3, name: NVIDIA GeForce

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


2022-12-04 07:58:49.224790: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-12-04 07:59:03.004562: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


391/391 [==============================] - 821s 2s/step - loss: 0.5729 - ACC: 0.7066 - ROC: 0.7181 - PR: 0.6104 - TP: 6130.0000 - TN: 29214.0000 - FP: 3039.0000 - FN: 11635.0000 - val_loss: 0.5214 - val_ACC: 0.7352 - val_ROC: 0.8045 - val_PR: 0.7055 - val_TP: 306.0000 - val_TN: 1629.0000 - val_FP: 68.0000 - val_FN: 629.0000


2022-12-04 08:13:07.636532: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221204_075828-hvy7k5tr/files/model-best/assets
Epoch 2/10
391/391 [==============================] - 805s 2s/step - loss: 0.4992 - ACC: 0.7549 - ROC: 0.8095 - PR: 0.7173 - TP: 8942.0000 - TN: 28815.0000 - FP: 3438.0000 - FN: 8823.0000 - val_loss: 0.4863 - val_ACC: 0.7717 - val_ROC: 0.8355 - val_PR: 0.7521 - val_TP: 638.0000 - val_TN: 1393.0000 - val_FP: 304.0000 - val_FN: 297.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221204_075828-hvy7k5tr/files/model-best/assets
Epoch 3/10
391/391 [==============================] - 815s 2s/step - loss: 0.4571 - ACC: 0.7836 - ROC: 0.8462 - PR: 0.7693 - TP: 10374.0000 - TN: 28822.0000 - FP: 3431.0000 - FN: 7391.0000 - val_loss: 0.4198 - val_ACC: 0.8043 - val_ROC: 0.8761 - val_PR: 0.8117 - val_TP: 572.0000 - val_TN: 1545.0000 - val_FP: 152.0000 - val_FN: 363.0000
INFO:tensorflow:Assets written to: /home/md

## End